In [35]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

from tensorflow.keras.layers import BatchNormalization

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df.drop('EIN', axis = 1, inplace = True)
df.drop('NAME', axis = 1, inplace = True)
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [15]:
# Look at APPLICATION_TYPE value counts for binning
counts = df['APPLICATION_TYPE'].value_counts()
counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = [index for index,row in counts.items() if row < 500]

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [17]:
# Look at CLASSIFICATION value counts for binning
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [18]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = df['CLASSIFICATION'].value_counts()
counts_greater_than_1 = classification_counts[classification_counts > 1]
counts_greater_than_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [19]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = [index for index,row in classification_counts.items() if row < classification_counts.mean()]

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
columns_to_convert = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
                      'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
processed_df = pd.get_dummies(df, columns=columns_to_convert, drop_first=True)
processed_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Split our preprocessed data into our features and target arrays
X = processed_df
y = processed_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Create the model with L2 regularization
nn_l2 = tf.keras.models.Sequential()

In [28]:
nn_l2.add(tf.keras.layers.Dense(units=10, activation="tanh", kernel_regularizer=l2(0.01), input_dim=38))
nn_l2.add(tf.keras.layers.Dense(units=8, activation="tanh", kernel_regularizer=l2(0.01)))
nn_l2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_l2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with L2 regularization
history_l2 = nn_l2.fit(X_train_scaled, y_train, epochs=20, batch_size=64)

Epoch 1/20
402/402 [==============================] - 1s 864us/step - loss: 0.4170 - accuracy: 0.9063
Epoch 2/20
402/402 [==============================] - 0s 842us/step - loss: 0.0953 - accuracy: 1.0000
Epoch 3/20
402/402 [==============================] - 0s 853us/step - loss: 0.0636 - accuracy: 1.0000
Epoch 4/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0517 - accuracy: 1.0000
Epoch 5/20
402/402 [==============================] - 0s 838us/step - loss: 0.0453 - accuracy: 0.9999
Epoch 6/20
402/402 [==============================] - 0s 992us/step - loss: 0.0405 - accuracy: 0.9999
Epoch 7/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0373 - accuracy: 0.9998
Epoch 8/20
402/402 [==============================] - 0s 985us/step - loss: 0.0342 - accuracy: 0.9998
Epoch 9/20
402/402 [==============================] - 0s 982us/step - loss: 0.0311 - accuracy: 0.9999
Epoch 10/20
402/402 [==============================] - 0s 863us/step - loss: 0.0295 - 

In [32]:
# Define a learning rate scheduler
def lr_schedule(epoch):
    lr = 0.001
    if epoch > 10:
        lr *= 0.1
    return lr

# Create the model with learning rate scheduler
nn_lr = tf.keras.models.Sequential()
nn_lr.add(tf.keras.layers.Dense(units=10, activation="tanh", input_dim=38))
nn_lr.add(tf.keras.layers.Dense(units=8, activation="tanh"))
nn_lr.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_lr.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

# Train the model with learning rate scheduler
history_lr = nn_lr.fit(X_train_scaled, y_train, epochs=20, batch_size=64, callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/20
402/402 [==============================] - 1s 874us/step - loss: 0.2526 - accuracy: 0.9322 - lr: 0.0010
Epoch 2/20
402/402 [==============================] - 0s 875us/step - loss: 0.0204 - accuracy: 0.9973 - lr: 0.0010
Epoch 3/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0074 - accuracy: 0.9986 - lr: 0.0010
Epoch 4/20
402/402 [==============================] - 0s 908us/step - loss: 0.0042 - accuracy: 0.9992 - lr: 0.0010
Epoch 5/20
402/402 [==============================] - 0s 934us/step - loss: 0.0027 - accuracy: 0.9997 - lr: 0.0010
Epoch 6/20
402/402 [==============================] - 0s 826us/step - loss: 0.0018 - accuracy: 0.9997 - lr: 0.0010
Epoch 7/20
402/402 [==============================] - 0s 990us/step - loss: 0.0011 - accuracy: 0.9999 - lr: 0.0010
Epoch 8/20
402/402 [==============================] - 0s 1ms/step - loss: 8.3934e-04 - accuracy: 0.9999 - lr: 0.0010
Epoch 9/20
402/402 [==============================] - 0s 1ms/step - loss: 7.6447

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_lr.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.9569e-04 - accuracy: 1.0000 - 256ms/epoch - 955us/step
Loss: 0.00019569242431316525, Accuracy: 1.0


In [38]:
# Create the model with batch normalization
nn_bn = tf.keras.models.Sequential()
nn_bn.add(tf.keras.layers.Dense(units=10, activation="tanh", input_dim=38))
nn_bn.add(BatchNormalization())
nn_bn.add(tf.keras.layers.Dense(units=8, activation="tanh"))
nn_bn.add(BatchNormalization())
nn_bn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

nn_bn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with batch normalization
history_bn = nn_bn.fit(X_train_scaled, y_train, epochs=20, batch_size=64)

Epoch 1/20
402/402 [==============================] - 1s 1ms/step - loss: 0.2683 - accuracy: 0.9312
Epoch 2/20
402/402 [==============================] - 1s 1ms/step - loss: 0.0239 - accuracy: 0.9963
Epoch 3/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 0.9986
Epoch 4/20
402/402 [==============================] - 1s 1ms/step - loss: 0.0047 - accuracy: 0.9991
Epoch 5/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 0.9994
Epoch 6/20
402/402 [==============================] - 1s 2ms/step - loss: 0.0021 - accuracy: 0.9993
Epoch 7/20
402/402 [==============================] - 1s 1ms/step - loss: 0.0019 - accuracy: 0.9993
Epoch 8/20
402/402 [==============================] - 1s 1ms/step - loss: 0.0015 - accuracy: 0.9995
Epoch 9/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.9997
Epoch 10/20
402/402 [==============================] - 0s 1ms/step - loss: 0.0011 - accuracy: 0.9996

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_bn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 2.2172e-04 - accuracy: 0.9999 - 260ms/epoch - 970us/step
Loss: 0.00022172130411490798, Accuracy: 0.9998833537101746
